In [2]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

periods = 12

realised = [66, 78, 57, 34, 5, 118, 76, 61, 23, 31, 68, 48]
forecast = [50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50]

demand = realised # case of prophecy, demand is known
#demand = forecast # case of frozen horizon, without updating new demand


initial_inv = 0

setupcost = 450
holdingcost = 2
backordercost = 20
#safetystock = 50
Big = sum(demand[t] for t in range(periods))

m = gp.Model("Prophet")

# Create variables
lotsize = {}
inventory ={}
setup = {}
for t in range(periods):
   lotsize[t] = m.addVar()
   inventory[t] = m.addVar(obj=holdingcost)
   setup[t] = m.addVar(vtype=GRB.BINARY, obj=setupcost)

# inventory balance
m.addConstr(inventory[0] == initial_inv+lotsize[0]-demand[0])
for t in range(1,periods):
   m.addConstr(inventory[t-1]+lotsize[t]-demand[t] == inventory[t])
    
# Logic constraints
for t in range(periods):
   m.addConstr(lotsize[t] <= Big*setup[t])

# add safety stock
# for t in range(horizon):
#    m.addConstr(inventory[t] >= safetystock)

m.optimize()

print("Objective: "+str(m.objVal))

v = m.getVars() # result of optimization problem


# new part => horizon planning
cost = 0
inv = initial_inv
for t in range(periods):
  if v[t*3+2].x > 0: #decision variable
    cost = cost+setupcost
  inv = inv+v[t*3].x-realised[t]
  print(v[3*t].x)
  if inv > 0:
     cost=cost+holdingcost*inv
  else: cost=cost-backordercost*inv  

print("Cost: "+str(cost))    
    
m.dispose()


#findings
# should not frozen too much periods, 
# could use safety stock => better than rolling horizon planning


Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 24 rows, 36 columns and 59 nonzeros
Model fingerprint: 0x7f11d9bf
Variable types: 24 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+02]
  Objective range  [2e+00, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+00, 1e+02]
Found heuristic solution: objective 5400.0000000
Presolve removed 3 rows and 4 columns
Presolve time: 0.00s
Presolved: 21 rows, 32 columns, 52 nonzeros
Variable types: 21 continuous, 11 integer (11 binary)

Root relaxation: objective 1.183338e+03, 12 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1183.33835    0    9 5400.00000 1183.33835  78.1%     -    0s
H    0     0                    4828.0000000 1183.33835  75.5%     -    0s
     0     0 2166.91189    0    8 4828.00000 2166.91189  55.1%   